In [123]:
import numpy as np
import pandas as pd

In [124]:
# def generate_outage_matrix(seed=42):
#     np.random.seed(seed)

#     hours_per_year = 365 * 24
#     generators = ["gas", "coal", "wind"]
#     outage_rates = {"gas": 100, "coal": 200, "wind": 400}
#     outage_durations = {"gas": 2, "coal": 3, "wind": 4}

#     matrix = np.zeros((len(generators), hours_per_year), dtype=bool)

#     for i, gen in enumerate(generators):
#         rate = outage_rates[gen]
#         duration = outage_durations[gen]
        
#         # Generate inter-arrival times (in hours) using exponential distribution
#         mean_interval = hours_per_year / rate
#         times = np.cumsum(np.random.exponential(mean_interval, rate * 2))
        
#         # Only keep outage start times within the year
#         outage_starts = times[times < hours_per_year].astype(int)
        
#         for start in outage_starts:
#             end = min(start + duration, hours_per_year)
#             matrix[i, start:end] = True

#     return matrix, generators


## Load data

In [ ]:
#year = snakemake.params.ty
year = 2028

#scenario = snakemake.params.op
scenario = 2

#technology_parameters = pd.read_hdf(snakemake.input.technology_parameters)
technology_parameters = pd.read_hdf("/trinity/home/fuhrand/ERAA/open_eraa/resources/technology_parameters.h5")

#plants = pd.read_hdf(snakemake.input.power_plants, "detailed")
plants = pd.read_hdf("/trinity/home/fuhrand/ERAA/open_eraa/resources/capacity_tables/individual_plants.h5", "detailed")

#maintanance_profile = pd.read_hdf(snakemake.input.maintenance_profiles, key="maintenance{}".format(year))
maintenance_profile = pd.read_hdf("/trinity/home/fuhrand/ERAA/open_eraa/resources/maintenance_profiles.h5", key="maintenance{}".format(year))

#save_hdf = snakemake.output.outage_patterns
save_hdf = "/trinity/home/fuhrand/ERAA/open_eraa/resources/outage_patterns/ty{}_op{}.h5".format(year, scenario)

# Prepare and quality check input data

In [126]:
outage_data = plants.join(technology_parameters[['forced_outage_share', 'forced_outage_n_days', 'maintenance_n_days']], on=['carrier', 'age_class'])
outage_data.forced_outage_n_days = outage_data.forced_outage_n_days.astype(int)

# Check if all plants have forced outage parameters
if outage_data[outage_data.forced_outage_share.isnull() | outage_data.forced_outage_n_days.isnull()].shape[0] != 0:
    raise ValueError(
                "There are plants with missing forced outage parameters!"
            )

In [ ]:
# Check if number of planned maintenance days match target vales in technology parameters

if outage_data[abs(outage_data['maintenance_n_days'].subtract(
    -maintenance_profile.sum().subtract(
        maintenance_profile.index.shape[0]).div(24))) > 5].shape[0] != 0:
    raise ValueError(
                "The number of maintenance days deviates by more than 5 from the target number!"
            )

/tmp/ipykernel_2011590/522949833.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if outage_data[abs(outage_data['maintenance_n_days'].subtract(


In [177]:
# Check if all generators have a cluster information attached

# if outage_data[outage_data.cluster.isnull()].shape[0] > 0:
#     raise ValueError(
#                 "The are generators without a cluster!"
#             )

## Simulate forced outages

In [ ]:
# Outages for single generator

key=[]

np.random.seed(year*scenario)

hours_per_year = len(maintenance_profile.index)
ls_generators = maintenance_profile.columns #[86, 95] #
np_forcedoutages = np.ones((len(ls_generators), hours_per_year), dtype=int)

for i, gen in enumerate(ls_generators):

    duration = outage_data.loc[gen].forced_outage_n_days * 24 # convert to hours
    num_outages_year = outage_data.loc[gen].forced_outage_share * hours_per_year / duration
    num_drawn_outages = int(num_outages_year * 2)
    mean_interval = hours_per_year/num_outages_year

    # Generate inter-arrival times (in hours) using exponential distribution
    times = np.cumsum(np.random.exponential(mean_interval, num_drawn_outages))

    # Only keep outage start times within the year
    outage_starts = times[times < hours_per_year].astype(int)

    # No forced outages during planned maintenance
    outage_starts = outage_starts[maintenance_profile[gen][outage_starts] != 0]

    for start in outage_starts:
        end = int(min(start + duration, hours_per_year))
        np_forcedoutages[i, start:end] = 0

    key.append((gen))

pd_unavailability_profiles = pd.DataFrame(np_forcedoutages, index=key).T
pd_unavailability_profiles.index.name = 'hour'

In [ ]:
# Outage profile for clusters as average outages weighted by generator capacity

pd_unavailability_profiles_cluster = pd_unavailability_profiles.multiply(
    outage_data.p_nom).T.groupby(outage_data.cluster).sum().T.div(
        outage_data.p_nom.groupby(outage_data.cluster).sum())

# Quality check and readout

In [163]:
# Check if outage rate does not deviate more than 5pp from target given in technology parameters 

if ((1 - pd_unavailability_profiles).sum().div(len(pd_unavailability_profiles)).groupby(
    outage_data.carrier).mean().subtract(
        outage_data.forced_outage_share.groupby(outage_data.carrier).mean()).abs() > 0.05).any():
    raise ValueError(
                "The outage rate deviates more than 5pp from target in technology parameter"
            )

In [167]:
pd_unavailability_profiles.to_hdf(save_hdf, key="single")
pd_unavailability_profiles_cluster.to_hdf(save_hdf, key="cluster")

/tmp/ipykernel_2011590/3376489462.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  pd_unavailability_profiles.to_hdf(save_hdf, key="single")
/tmp/ipykernel_2011590/3376489462.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_items] [items->None]

  pd_unavailability_profiles.to_hdf(save_hdf, key="single")


In [182]:
testhdf = pd.read_hdf(save_hdf, key="cluster")
testhdf

cluster,AL00 CCGT entry 2030,AL00 OCGT entry 2030,AT00 CCGT exit 2030,AT00 CCGT exit 2033,AT00 CCGT exit 2035,AT00 CCGT exit 2036,AT00 OCGT exit 2030,AT00 OCGT exit 2033,AT00 OCGT exit 2035,AT00 OCGT exit 2036,...,UK00 nuclear entry 2035,UK00 nuclear exit 2030,UK00 nuclear exit 2036,UK00 oil exit 2030,UK00 oil exit 2033,UK00 oil exit 2036,UKNI CCGT exit 2036,UKNI OCGT exit 2036,UKNI biomass exit 2036,UKNI oil exit 2036
hour,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,...,0.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000
1,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,...,0.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000
2,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,...,0.0,1.0,1.0,1.0,0.747475,1.0,1.0,1.0,1.0,1.000000
3,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,...,0.0,1.0,1.0,1.0,0.747475,1.0,1.0,1.0,1.0,1.000000
4,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,...,0.0,1.0,1.0,1.0,0.747475,1.0,1.0,1.0,1.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.005662,1.0,...,0.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,0.871795
8756,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.005662,1.0,...,0.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,0.871795
8757,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,...,0.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,0.743590
